In [1]:
import random
import torch
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from timeit import default_timer

sys.path.append('../')
from models import FNN3d, FNN_train


# Unit test with f -> 2f

In [2]:
torch.manual_seed(0)
np.random.seed(0)

n_data = 2048
Lx, Ly, Lz, Lt = 2*np.pi, 2*np.pi, 2*np.pi, 1.0
nx, ny, nz, nt = 8, 16, 32, 64
input_u = np.random.normal(0, 1, (n_data, nx, ny, nz, nt))
output_u = 2.0*input_u 

grid = np.zeros((nx, ny, nz, nt, 4))
grid_x, grid_y, grid_z, grid_t = np.linspace(0, Lx, nx), np.linspace(0, Ly, ny), np.linspace(0, Lz, nz), np.linspace(0, Lt, nt)
grid[:,:,:,:,0], grid[:,:,:,:,1], grid[:,:,:,:,2], grid[:,:,:,:,3] = np.swapaxes(np.meshgrid(grid_x, grid_y, grid_z, grid_t), 1, 2)
                        

In [3]:
downsample_ratio = 1

L, Ne_ref = 1.0, 2**12
Ne = Ne_ref//downsample_ratio

n_train = n_test = 2**10
x_train = torch.from_numpy(np.concatenate((input_u[0:n_train, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, np.newaxis], np.tile(grid, (n_train,1,1,1,1,1))), axis=-1).astype(np.float32))
y_train = torch.from_numpy(output_u[0:n_train, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, np.newaxis].astype(np.float32))
# x_train, y_train are [n_data, n_x, n_channel] arrays
x_test = torch.from_numpy(np.concatenate((input_u[-n_test:, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, np.newaxis], np.tile(grid, (n_test,1,1,1,1,1))), axis=-1).astype(np.float32))
y_test = torch.from_numpy(output_u[-n_test:, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, 0::downsample_ratio, np.newaxis].astype(np.float32))
# x_test, y_test are [n_data, n_x, n_channel] arrays


n_fno_layers = 3
k_max = 8
d_f = 32
# fourier k_max
modes = [k_max] * n_fno_layers
# channel d_f
layers = [d_f] * (n_fno_layers + 1)
fc_dim = d_f
in_dim = 1+4
out_dim = 1
act = "gelu"

base_lr = 0.001
epochs = 1000
# scheduler = "CosineAnnealingLR"
weight_decay = 1e-4

scheduler = "MultiStepLR"

pad_ratio = 0.05

milestones = [200, 300, 400, 500, 800,900]
scheduler_gamma = 0.5
batch_size=2
normalization_x = True
normalization_y = True
normalization_dim = []


config = {"model" : {"modes": modes, "fc_dim": fc_dim, "layers": layers, "in_dim": in_dim, "out_dim":out_dim, "act": act, "pad_ratio":pad_ratio},
          "train" : {"base_lr": base_lr, "weight_decay": weight_decay, "epochs": epochs, "scheduler": scheduler, "milestones": milestones, "scheduler_gamma": scheduler_gamma, "batch_size": batch_size, 
                    "normalization_x": normalization_x,"normalization_y": normalization_y, "normalization_dim": normalization_dim}}




In [ ]:
start = default_timer()
train_rel_l2_losses, test_rel_l2_losses, test_l2_losses, cost = FNN_train(x_train, y_train, x_test, y_test, config, save_model_name="models/test_FNO")
end = default_timer()
print("epochs = ", epochs, "elapsed time = ", end - start)

Epoch :  0  Rel. Train L2 Loss :  0.09282728047037381  Rel. Test L2 Loss :  0.022883910736709367  Test L2 Loss :  0.04576989990164293
